In [ ]:
!pip install pandas scipy numpy matplotlib ipywidgets scikit-learn seaborn scipy statsmodels 

In [2]:
import glob
import pandas as pd
import os
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, fixed #slider for the radar plots
from sklearn.preprocessing import MinMaxScaler #for keeping the same scales for the radar plots
import seaborn as sns
from scipy.stats import pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [5]:
'''INSERT THE MATRICES THAT HAVE THE MSSD DATA AND REGIONAL DATA'''

region_data = pd.read_csv('/projects/tsecara/SPINS_ASD_Project2/Glasser_Tian_roi_info.csv')
mssd_data = pd.read_csv('/projects/tsecara/SPINS_ASD_Project2/data/combined/regional_MSSD/neurocombat/combined_mssd_mean_EA_RScleanFINAL.csv', header = 0, index_col = 0) #for changing this when the rows and columns are a little funky

'''HAVE TO MAKE SURE THAT THE REGION LABELS ARE ACTUALLY WITHIN THE HEADER INSTEAD OF JUST THE FIRST ROW'''

'HAVE TO MAKE SURE THAT THE REGION LABELS ARE ACTUALLY WITHIN THE HEADER INSTEAD OF JUST THE FIRST ROW'

In [6]:
'''CREATES A DICTIONARY TO HELP WITH MAPPING THE NETWORKS TOGETHER'''
# Create a mapping from region names to their corresponding network
region_to_network = dict(zip(region_data['atlas_roi'], region_data['net_ColeAnt']))

In [7]:
'''TO SET UP THE RADAR PLOTS AND THE INFORMATION THAT GOES WIHTIN IT'''

# Get unique networks
networks = region_data['net_ColeAnt'].unique()

# Initializes DataFrame to hold the all the MSSD values by network instead of region
network_mssd_data = pd.DataFrame(index=mssd_data.index, columns=networks)

# Iterate over each participant
for participant in mssd_data.index:
    # Get the MSSD values for the participant
    participant_mssd = mssd_data.loc[participant]
    
    # Initialize a dictionary to hold all of the MSSD values for each network
    network_mssd = {network: [] for network in networks}
    
    # Group the MSSD values by network (by looking at their regions and how they correspond to the same network)
    for region, mssd_value in participant_mssd.items():
        if region in region_to_network:
            network = region_to_network[region]
            network_mssd[network].append(mssd_value)
    
    # Calculate the mean MSSD value for each network
    for network in networks:
        if network_mssd[network]:
            network_mssd_data.loc[participant, network] = np.mean(network_mssd[network])
        else:
            network_mssd_data.loc[participant, network] = np.nan

# Drop any columns (networks) that are entirely NaN - which I hope is none...
network_mssd_data.dropna(axis=1, how='all', inplace=True)


In [ ]:
#Saves a csv file containing participant network MSSD values 
pd.DataFrame(network_mssd_data).to_csv("/projects/tsecara/SPINS_ASD_Project2/data/combined/network_MSSD/EA_MSSD_NETWORK.csv") #gets a csv file here